In [1]:
from tensorflow.keras import models
import numpy as np
import pickle

In [2]:
model = models.load_model("model.h5")
with open("dicts.tuple","rb") as fb:
    input_token_index,target_token_index,max_encoder_seq_length,num_encoder_tokens = pickle.load(fb)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [3]:
def encode_input(input_text):  
    encoder_input_data = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    for t, char in enumerate(input_text.split()):
        if char in input_token_index.keys():
            encoder_input_data[0, t, input_token_index[char]] = 1.
        else:
            encoder_input_data[0, t, input_token_index["<unk>"]] = 1.
    encoder_input_data[0, t + 1:, input_token_index['<end>']] = 1.
    return encoder_input_data

def predict(line):
    s = line
    e = encode_input(s)
    prediction = model.predict(e)
    ans = ""
    for p in prediction[0]:
        out = reverse_target_char_index[np.argmax(p)]
        if out != "<end>" and out != "<start>":
            if out.isnumeric():
                ans += s.split()[int(out,base=10)-1]+" "
            else:
                ans += str(out)+" "
    return ans.strip()

In [5]:
code = [
    "let a to one",
    "initialize b to two",
    "if a is greater than b",
    "set c to addition of a and b",
    "end if"
]
for c in code:
    print(predict(c))

assign a <exp>1
assign b <exp>2
if greater a b do
sub a b equ c
endif
